In [1]:
import os
import datetime

from vllm.steer_vectors.request import SteerVectorRequest
from vllm import LLM, SamplingParams

from easysteer.steer import StatisticalControlVector
from transformers import AutoTokenizer
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

/home/meixinyu/anaconda3/envs/easysteer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-13 22:14:43 [__init__.py:244] Automatically detected platform cuda.


In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"
)

In [3]:
model_path = "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"
# vector_path="/home/meixinyu/EasySteer/vectors/thinking_switch_pca_MATH-500.gguf" #change to your path
vector_path="MATH500.gguf" #change to your path

control_vector = StatisticalControlVector.import_gguf(vector_path)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='pca', directions={0: memmap([-0.01113699, -0.06029188,  0.04215517, ..., -0.01189059,
        -0.02577175, -0.01083255], shape=(1536,), dtype=float32), 1: memmap([ 0.02075267, -0.00759126,  0.01752456, ..., -0.01258039,
        -0.03163083, -0.04065328], shape=(1536,), dtype=float32), 2: memmap([ 0.0100673 , -0.0234082 ,  0.02484847, ...,  0.06437913,
        -0.01141085, -0.05111764], shape=(1536,), dtype=float32), 3: memmap([-0.01157416, -0.00161835,  0.04929598, ...,  0.02504394,
        -0.02055443, -0.04532089], shape=(1536,), dtype=float32), 4: memmap([ 0.00013692,  0.01033693,  0.02944114, ...,  0.03095547,
        -0.03417241, -0.00300148], shape=(1536,), dtype=float32), 5: memmap([-0.00319834,  0.04855272,  0.02122804, ..., -0.01298449,
        -0.04043267, -0.03302082], shape=(1536,), dtype=float32), 6: memmap([ 0.01014982,  0.00519793,  0.03920706, ..., -0.01955185,
         0.04191752,  0.01931267], shape=(1536,), dtype

In [4]:
steer_vector_request_pos = SteerVectorRequest(
            steer_vector_name="thinking_switch_pos",
            steer_vector_id=1,
            steer_vector_local_path=vector_path,
            scale=2.0,
            prefill_trigger_tokens=[-1],
            prefill_trigger_positions=[-1],
            generate_trigger_tokens=[-1],
            debug=False,
            algorithm='direct'
        )
steer_vector_request_neg = SteerVectorRequest(
            steer_vector_name="thinking_switch_neg",
            steer_vector_id=2,
            steer_vector_local_path=vector_path,
            scale=-2.0,
            prefill_trigger_tokens=[-1],
            prefill_trigger_positions=[-1],
            generate_trigger_tokens=[-1],
            debug=False,
            algorithm='direct'
        )

In [5]:
sampling_params = SamplingParams(temperature=0.0,max_tokens=4096)
prompt_template = "<|im_start|>user\n%s<|im_end|>\n<|im_start|>assistant\n"
prompt = "Find the greatest common divisor of $3339$, $2961$, and $1491$."
# prompt = "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now."
input = prompt_template % prompt

In [6]:
llm = LLM(model=model_path, enable_steer_vector=True, tensor_parallel_size=1,enforce_eager=True)
output_base = llm.generate(
    input,
    sampling_params
)
generated_text_base = output_base[0].outputs[0].text

output_pos = llm.generate(
                input,
                sampling_params,
                steer_vector_request=steer_vector_request_pos
            )
generated_text_pos = output_pos[0].outputs[0].text

output_neg = llm.generate(
                input,
                sampling_params,
                steer_vector_request=steer_vector_request_neg
            )
generated_text_neg = output_neg[0].outputs[0].text

INFO 07-13 22:14:52 [config.py:841] This model supports multiple tasks: {'classify', 'reward', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 07-13 22:14:52 [config.py:1472] Using max model len 32768
WARNING 07-13 22:14:52 [arg_utils.py:1770] --enable-steer-vector is not supported by the V1 Engine. Falling back to V0. 


2025-07-13 22:14:53,082	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 07-13 22:14:53 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-13 22:14:53 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observabi

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.08it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.08it/s]


INFO 07-13 22:14:56 [default_loader.py:272] Loading weights took 1.03 seconds
INFO 07-13 22:14:56 [model_runner.py:1255] Model loading took 2.8876 GiB and 1.185043 seconds


INFO 07-13 22:14:58 [worker.py:295] Memory profiling takes 1.42 seconds
INFO 07-13 22:14:58 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-13 22:14:58 [worker.py:295] model weights take 2.89GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 2.05GiB; the rest of the memory reserved for KV Cache is 37.68GiB.
INFO 07-13 22:14:58 [executor_base.py:115] # cuda blocks: 88185, # CPU blocks: 9362
INFO 07-13 22:14:58 [executor_base.py:120] Maximum concurrency for 32768 tokens per request: 43.06x
INFO 07-13 22:15:03 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 6.49 seconds


Processed prompts: 100%|██████████| 1/1 [00:12<00:00, 12.66s/it, est. speed input: 2.61 toks/s, output: 32.79 toks/s]


In [7]:
print(generated_text_base)

To find the greatest common divisor (GCD) of the numbers 3339, 2961, and 1491, we will use the Euclidean algorithm. The Euclidean algorithm involves repeated division and taking remainders until we reach a remainder of 0. The last non-zero remainder is the GCD.

First, we find the GCD of 3339 and 2961.

1. Divide 3339 by 2961 and find the remainder:
   \[
   3339 = 2961 \cdot 1 + 378
   \]
   So, the remainder is 378.

2. Now, we need to find the GCD of 2961 and 378. Divide 2961 by 378:
   \[
   2961 = 378 \cdot 7 + 255
   \]
   So, the remainder is 255.

3. Next, we find the GCD of 378 and 255. Divide 378 by 255:
   \[
   378 = 255 \cdot 1 + 123
   \]
   So, the remainder is 123.

4. Now, we find the GCD of 255 and 123. Divide 255 by 123:
   \[
   255 = 123 \cdot 2 + 0
   \]
   So, the remainder is 0. The last non-zero remainder is 123.

Thus, the GCD of 3339 and 2961 is 123.

Next, we need to find the GCD of 123 and 1491.

1. Divide 1491 by 123:
   \[
   1491 = 123 \cdot 12 + 15
   \

In [8]:
print(generated_text_pos)

To find the greatest common divisor (GCD) of the numbers 3339, 2961, and 1491, we can use the Euclidean algorithm, which involves repeated division and taking remainders. Let's start by finding the GCD of 3339 and 2961, and then use that result to find the GCD with 1491.

First, we find the GCD of 3339 and 2961 using the Euclidean algorithm:
1. Divide 3339 by 2961 and find the remainder:
   \[
   3339 = 2961 \cdot 1 + 378
   \]
   So, the remainder is 378.
2. Now, divide 2961 by 378:
   \[
   2961 = 378 \cdot 7 + 285
   \]
   So, the remainder is 285.
3. Next, divide 378 by 285:
   \[
   378 = 285 \cdot 1 + 93
   \]
   So, the remainder is 93.
4. Now, divide 285 by 93:
   \[
   285 = 93 \cdot 3 + 18
   \]
   So, the remainder is 18.
5. Finally, divide 93 by 18:
   \[
   93 = 18 \cdot 5 + 3
   \]
   So, the remainder is 3.
6. Now, divide 18 by 3:
   \[
   18 = 3 \cdot 6 + 0
   \]
   So, the remainder is 0.

Since the remainder is now 0, the GCD of 3339 and 2961 is the last non-zero rema

In [9]:
print(generated_text_neg)

To find the greatest common divisor (GCD) of the numbers 3339, 2961, and 1491, we will use the Euclidean algorithm. The Euclidean algorithm involves repeated division and taking remainders until we reach a remainder of zero. The last non-zero remainder is the GCD.

First, we find the GCD of 3339 and 2961. We start by dividing 3339 by 2961 and finding the remainder:
\[3339 = 2961 \cdot 1 + 378.\]
So, the remainder is 378. Next, we divide 2961 by 378:
\[2961 = 378 \cdot 7 + 255.\]
So, the remainder is 255. Next, we divide 378 by 255:
\[378 = 255 \cdot 1 + 123.\]
So, the remainder is 123. Next, we divide 255 by 123:
\[255 = 123 \cdot 2 + 0.\]
So, the remainder is 0. Therefore, the GCD of 3339 and 2961 is 123.

Next, we need to find the GCD of 123 and 1491. We start by dividing 1491 by 123:
\[1491 = 123 \cdot 12 + 0.\]
So, the remainder is 0. Therefore, the GCD of 123 and 1491 is 123.

Thus, the GCD of 3339, 2961, and 1491 is \(\boxed{123}\).


In [10]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename=f'fractreason_GSM8K_{timestamp}.txt'
# filename=f'../../temp/fractreason_GSM8K_{timestamp}.txt'
with open(filename, 'w', encoding='utf-8') as f:
    f.write("=== Base ===\n")
    f.write(generated_text_base + "\n\n")

    f.write("=== Positive: Slow thinking ===\n")
    f.write(generated_text_pos + "\n\n")

    f.write("=== Negative: Fast thinking ===\n")
    f.write(generated_text_neg + "\n")


1419
1249
952
